<a href="https://colab.research.google.com/github/sqhang/githubTest/blob/main/keras_tuner_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Source and Copyright
This Google Colab notebook was adapted from one tutorial from Keras Tuner. The notebook changes the tuning of the original ANN model to a new CNN model and adds several more functionalities.

This notebook is just used for internal learning and nonprofit purposes. All copyrights are reserved for TensorFlow authors.

# Introduction to the Keras Tuner

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/keras_tuner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called *hyperparameter tuning* or *hypertuning*.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:
1. **Model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Flatten, LeakyReLU, ReLU, Dropout
import numpy as np

Install and import the Keras Tuner.

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 5.4MB/s 


In [ ]:
import kerastuner as kt

## Download and prepare the dataset

In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist).

Load the data.

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


Scale the data.

In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

Reshape the data for CNN training

In [ ]:
# Reshape the dataset for the CNN 
img_train = img_train[..., np.newaxis]
img_test = img_test[..., np.newaxis]
img_train.shape

(60000, 28, 28, 1)

## Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a *hypermodel*.

You can define a hypermodel through two approaches:

* By using a model builder function
* By subclassing the `HyperModel` class of the Keras Tuner API

You can also use two pre-defined `HyperModel` classes - [HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperxception-class) and [HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class) for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [ ]:
def build_model(hp):
  '''
  This function takes in hp, a hyperparameter class instance, and outputs a CNN model
  '''
  model = Sequential()

  # for i in range(hp.Int('conv_blocks', 3, 5, default=3))
  # Tune the number of units in the first Conv layer
  # Choose an optimal value between 32-256
  hp_units1 = hp.Int('units_conv_layer1', min_value=32, max_value=256, step=32)

  # Tune the alpha value for the LeakyReLu activation in the first Conv layer
  # Choose between 0.3 and 0.0 {0.3: default for LeakyReLu, 0.0: equivalent to ReLU)
  hp_alpha1 = hp.Choice('alpha_conv_layer1', values = [0.3, 0.0])
  model.add(Conv2D(filters = hp_units1, kernel_size=(3,3), padding = 'same', activation=LeakyReLU(alpha=hp_alpha1), input_shape = (28, 28, 1)))

  # Tune the number of units in the second Conv layer
  # Choose an optimal value between 32-256
  hp_units2 = hp.Int('units_conv_layer2', min_value=32, max_value=256, step=32)

  # Tune the alpha value for the LeakyReLu activation in the second Conv layer
  # Choose between 0.3 and 0.0 {0.3: default for LeakyReLu, 0.0: equivalent to ReLU)
  hp_alpha2 = hp.Choice('alpha_conv_layer2', values = [0.3, 0.0])
  model.add(Conv2D(filters=hp_units2, kernel_size=(3,3), activation=LeakyReLU(alpha=hp_alpha2)))

  model.add(MaxPooling2D((3,3)))

  model.add(Flatten())

  # Introduce dropout to surpress overfitting
  model.add(Dropout(0.3))

  model.add(Dense(10))

  # Tune the learning rate for the Adam optimizer
  # Choose an optimal value from 0.01, or 0.001 (default of Adam).
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3])

  # from_logits = True since not adding softmax yet
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),  
                metrics=['accuracy'])
  
  return model


## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. In this tutorial, you use the [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

In [ ]:
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')
objective= kt.Objective("prec_class1", direction="max"),

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [ ]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print("The hyperparameter search is complete.")

# In the future, to better clarify the best hp, can uncomment and modify these print statements
'''
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")
'''

Trial 30 Complete [00h 01m 56s]
val_accuracy: 0.9073333144187927

Best val_accuracy So Far: 0.9195833206176758
Total elapsed time: 00h 27m 38s
INFO:tensorflow:Oracle triggered exit
The hyperparameter search is complete.


'\nprint(f"""\nThe hyperparameter search is complete. The optimal number of units in the first densely-connected\nlayer is {best_hps.get(\'units\')} and the optimal learning rate for the optimizer\nis {best_hps.get(\'learning_rate\')}.\n""")\n'

## Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [ ]:
# Show a summary of the tuner search
# By default, shows 10 best trials
tuner.results_summary()

Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units_conv_layer1: 160
alpha_conv_layer1: 0.0
units_conv_layer2: 256
alpha_conv_layer2: 0.0
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 962c4be7e5a8ec34c2180c3ee54dbd19
Score: 0.9195833206176758
Trial summary
Hyperparameters:
units_conv_layer1: 192
alpha_conv_layer1: 0.0
units_conv_layer2: 96
alpha_conv_layer2: 0.0
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 754b61683c0d591877b14cab20c18281
Score: 0.9193333387374878
Trial summary
Hyperparameters:
units_conv_layer1: 96
alpha_conv_layer1: 0.0
units_conv_layer2: 192
alpha_conv_layer2: 0.3
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 5fd6ee9758fa141c5e0aa42832c75e0b
Score: 0.9192500114440918
Trial summar

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 20 epochs
# Find the suitable number of epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=20, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/20
1500/1500 [==============================] - 16s 10ms/step - loss: 0.3977 - accuracy: 0.8583 - val_loss: 0.3037 - val_accuracy: 0.8892
Epoch 2/20
1500/1500 [==============================] - 16s 10ms/step - loss: 0.2821 - accuracy: 0.8980 - val_loss: 0.2875 - val_accuracy: 0.8947
Epoch 3/20
1500/1500 [==============================] - 16s 11ms/step - loss: 0.2407 - accuracy: 0.9134 - val_loss: 0.2518 - val_accuracy: 0.9091
Epoch 4/20
1500/1500 [==============================] - 16s 10ms/step - loss: 0.2076 - accuracy: 0.9247 - val_loss: 0.2340 - val_accuracy: 0.9153
Epoch 5/20
1500/1500 [==============================] - 15s 10ms/step - loss: 0.1854 - accuracy: 0.9324 - val_loss: 0.2318 - val_accuracy: 0.9155
Epoch 6/20
1500/1500 [==============================] - 15s 10ms/step - loss: 0.1688 - accuracy: 0.9383 - val_loss: 0.2375 - val_accuracy: 0.9147
Epoch 7/20
1500/1500 [==============================] - 15s 10ms/step - loss: 0.1483 - accuracy: 0.9455 - val_loss: 0.2367 -

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/13
1500/1500 [==============================] - 16s 10ms/step - loss: 0.4005 - accuracy: 0.8588 - val_loss: 0.3009 - val_accuracy: 0.8894
Epoch 2/13
1500/1500 [==============================] - 16s 10ms/step - loss: 0.2809 - accuracy: 0.8991 - val_loss: 0.2665 - val_accuracy: 0.9036
Epoch 3/13
1500/1500 [==============================] - 16s 11ms/step - loss: 0.2420 - accuracy: 0.9131 - val_loss: 0.2447 - val_accuracy: 0.9117
Epoch 4/13
1500/1500 [==============================] - 16s 10ms/step - loss: 0.2123 - accuracy: 0.9226 - val_loss: 0.2353 - val_accuracy: 0.9140
Epoch 5/13
1500/1500 [==============================] - 15s 10ms/step - loss: 0.1876 - accuracy: 0.9317 - val_loss: 0.2505 - val_accuracy: 0.9130
Epoch 6/13
1500/1500 [==============================] - 15s 10ms/step - loss: 0.1701 - accuracy: 0.9375 - val_loss: 0.2464 - val_accuracy: 0.9113
Epoch 7/13
1500/1500 [==============================] - 15s 10ms/step - loss: 0.1530 - accuracy: 0.9448 - val_loss: 0.2244 -

To finish this tutorial, evaluate the hypermodel on the test data.

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 2s 5ms/step - loss: 0.2939 - accuracy: 0.9161
[test loss, test accuracy]: [0.2939131557941437, 0.916100025177002]


The test accuracy result is around 2% higher than the test accuracy reported in the original Keras Tuner guide.

The `my_dir/intro_to_kt` directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite=True` argument while instantiating the tuner.

## Summary

In this tutorial, you learned how to use the Keras Tuner to tune hyperparameters for a model. To learn more about the Keras Tuner, check out these additional resources:

* [Keras Tuner on the TensorFlow blog](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* [Keras Tuner website](https://keras-team.github.io/keras-tuner/)

To see additional tutorials and examples of the Keras Tuner, check out:

*  [Google I/O'19 Keras Tuner](https://www.youtube.com/watch?v=Un0JDL3i5Hg)
*  [Hands on hyperparameter tuning with Keras Tuner](https://www.sicara.ai/blog/hyperparameter-tuning-keras-tuner)

Also check out the [HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) in TensorBoard to interactively tune your model hyperparameters.